### Get scripts

In [0]:
!wget -q https://raw.githubusercontent.com/Mmiglio/SpeechRecognition/master/src/getData.py
!wget -q https://raw.githubusercontent.com/Mmiglio/SpeechRecognition/master/src/constants.py

### Download speech data

In [0]:
from getData import downloadSpeechData, getDataDict

In [5]:
downloadSpeechData(data_path='speechData/')

Extracting /content/speechData/train.tar.gz into /content/speechData/train
Extracting /content/speechData/test.tar.gz into /content/speechData/test


In [0]:
dataDict = getDataDict(data_path='speechData/')

In [8]:
len(dataDict['train']['files'])

51094